In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
import warnings
import itertools
import pickle
from numpy import linalg as LA
import random
warnings.filterwarnings('ignore')

In [3]:
test_redu_path = 'https://raw.githubusercontent.com/mariabandeira/Reconhecimento-de-Digitos/main/dataset_redu/test_redu.csv'
train_redu_path = 'https://raw.githubusercontent.com/mariabandeira/Reconhecimento-de-Digitos/main/dataset_redu/train_redu.csv'

train_redu = pd.read_csv(train_redu_path, sep=';')
test_redu = pd.read_csv(test_redu_path, sep=';')

# Melhor sequência de dígitos

In [4]:
# normalização para PLA e Regressão Logística
scaler = RobustScaler()

train_redu_scaled = scaler.fit_transform(train_redu.drop('label', axis=1)) # ajusta e transforma
test_redu_scaled = scaler.transform(test_redu.drop('label', axis=1)) # apenas transforma (faz com que não haja vazamento de dados)

train_redu['i_normalizada'] = train_redu_scaled[:,0]
train_redu['s_normalizada'] = train_redu_scaled[:,1]

test_redu['i_normalizada'] = test_redu_scaled[:,0]
test_redu['s_normalizada'] = test_redu_scaled[:,1]

In [5]:
class LogisticRegression_:
  def __init__(self, eta=0.1, tmax=1000, bs=20):
    self.eta = eta
    self.tmax = tmax
    self.batch_size = bs

  # Infere o vetor w da funçao hipotese
  #Executa a minimizao do erro de entropia cruzada pelo algoritmo gradiente de descida
  def fit(self, _X, _y):
    X = np.array(_X)
    y = np.array(_y)

    N = X.shape[0]
    d = X.shape[1]
    w = np.zeros(d, dtype=float)
    self.w = []

    for t in range(self.tmax):
        vsoma = np.zeros(d, dtype=float)
      
        # Seleciona um lote de dados aleatorios
        if self.batch_size < N:
            idx = random.sample(range(N), self.batch_size)
            batchX = [X[i] for i in idx]
            batchY = [y[i] for i in idx]
        else:
            batchX = X
            batchY = y

        # Calcula o gradiente da funcao de erro
        for xn, yn in zip(batchX, batchY):
            vsoma += (yn * xn) / (1 + np.exp((yn * w).T @ xn))

        grad_t = vsoma / len(batchY)
        # condição de parada
        if LA.norm(grad_t) < 0.0001:
            break

        w = w + (self.eta * grad_t)

    self.w = w

  #funcao hipotese inferida pela regressa logistica
  def predict_prob(self, X):
    s = np.dot(X, self.w)
    prob = np.exp(s) / (1 + np.exp(s))
    return prob

  #Predicao por classificação linear
  def predict(self, X):
    prob = self.predict_prob(X)
    y = np.where(prob >= 0.5, 1, -1)
    return y

  def getW(self):
    return self.w

  def getRegressionY(self, regressionX, shift=0):
    return (-self.w[0]+shift - self.w[1]*regressionX) / self.w[2]

In [6]:
def UmXtodos(train_redu, test_redu, digits=[0, 1, 4, 5]):
    # Normalize train_redu
    scaler = RobustScaler()
    train_redu_scaled = scaler.fit_transform(train_redu.drop('label', axis=1))
    train_redu['i_normalizada'] = train_redu_scaled[:, 0]
    train_redu['s_normalizada'] = train_redu_scaled[:, 1]

    test_redu_scaled = scaler.transform(test_redu.drop('label', axis=1))
    test_redu['i_normalizada'] = test_redu_scaled[:, 0]
    test_redu['s_normalizada'] = test_redu_scaled[:, 1]

    modelosParent = []

    def classificar_digito(modelos, x):
        for modelo in modelos:
            if modelo['modelo'].predict([x])[0] == 1:
                return modelo['digito']
        return 5

    for d in digits:
        modelosChild = {'digito': d}

        X_train = np.hstack((np.ones((len(train_redu), 1)), train_redu[['i_normalizada', 's_normalizada']].values))
        y_train = np.array([1 if y_ == d else -1 for y_ in train_redu['label']])

        X_test = np.hstack((np.ones((len(test_redu), 1)), test_redu[['i_normalizada', 's_normalizada']].values))
        y_test = np.array([1 if y_ == d else -1 for y_ in test_redu['label']])

        modelo = LogisticRegression_(eta=0.1, tmax=1000, bs=20)
        modelo.fit(X_train, y_train)

        modelosChild['modelo'] = modelo
        modelosParent.append(modelosChild)

    for modelosChild in modelosParent:
        modelo = modelosChild['modelo']
        digit = modelosChild['digito']

        train_redutemp = train_redu[train_redu['label'] == digit]
        test_redutemp = test_redu[test_redu['label'] == digit]

        X_train = np.hstack((np.ones((len(train_redutemp), 1)), train_redutemp[['i_normalizada', 's_normalizada']].values))
        y_train = np.array([1 if y_ == digit else -1 for y_ in train_redutemp['label']])

        X_test = np.hstack((np.ones((len(test_redutemp), 1)), test_redutemp[['i_normalizada', 's_normalizada']].values))
        y_test = np.array([1 if y_ == digit else -1 for y_ in test_redutemp['label']])

        # calcular ein e eout
        eIn = sum(1 for i in range(len(X_train)) if classificar_digito(modelosParent, X_train[i]) != train_redutemp['label'].iloc[i])
        eOut = sum(1 for i in range(len(X_test)) if classificar_digito(modelosParent, X_test[i]) != test_redutemp['label'].iloc[i])

        modelosChild['eIn'] = eIn / len(X_train)
        modelosChild['eOut'] = eOut / len(X_test)

    return modelosParent

In [8]:
digits = [0, 1, 4, 5]
list(itertools.permutations(digits))

[(0, 1, 4, 5),
 (0, 1, 5, 4),
 (0, 4, 1, 5),
 (0, 4, 5, 1),
 (0, 5, 1, 4),
 (0, 5, 4, 1),
 (1, 0, 4, 5),
 (1, 0, 5, 4),
 (1, 4, 0, 5),
 (1, 4, 5, 0),
 (1, 5, 0, 4),
 (1, 5, 4, 0),
 (4, 0, 1, 5),
 (4, 0, 5, 1),
 (4, 1, 0, 5),
 (4, 1, 5, 0),
 (4, 5, 0, 1),
 (4, 5, 1, 0),
 (5, 0, 1, 4),
 (5, 0, 4, 1),
 (5, 1, 0, 4),
 (5, 1, 4, 0),
 (5, 4, 0, 1),
 (5, 4, 1, 0)]

In [11]:
# Lista de dígitos
digits = [0, 1, 4, 5]

# Gerar todas as permutações
permutations = list(itertools.permutations(digits))

# Inicializar variáveis para armazenar os melhores resultados
best_permutation = None
best_Ein_min = float('inf')
best_Eout_min = float('inf')
best_modelos = None

# Testar todas as permutações
for index, perm in enumerate(permutations):
    print(f"Permutação {index + 1}/{len(permutations)}")
    print(f"Permutação: {perm}")
    modelos = UmXtodos(train_redu, test_redu, perm)
    Eins = [m['eIn'] for m in modelos]
    Eouts = [m['eOut'] for m in modelos]
    
    # medias
    Ein_mean = np.mean(Eins)
    Eout_mean = np.mean(Eouts)

    # Verificar se é a melhor permutação
    if Ein_mean < best_Ein_min:
        best_Ein_min = Ein_mean
        best_Eout_min = Eout_mean
        best_permutation = perm
        best_modelos = modelos

    print(f"E_in médio: {Ein_mean}")
    print(f"E_out médio: {Eout_mean}")
    print("")

# Exibir os melhores resultados
print(f"Melhor permutação: {best_permutation}")
print(f"Melhor E_in mínimo: {best_Ein_min}")
print(f"Melhor E_out mínimo: {best_Eout_min}")

Permutação 1/24
Permutação: (0, 1, 4, 5)
E_in médio: 0.2605882352941176
E_out médio: 0.2580275229357798

Permutação 2/24
Permutação: (0, 1, 5, 4)
E_in médio: 0.26294117647058823
E_out médio: 0.2603211009174312

Permutação 3/24
Permutação: (0, 4, 1, 5)
E_in médio: 0.2605882352941176
E_out médio: 0.26146788990825687

Permutação 4/24
Permutação: (0, 4, 5, 1)
E_in médio: 0.26588235294117646
E_out médio: 0.26146788990825687

Permutação 5/24
Permutação: (0, 5, 1, 4)
E_in médio: 0.2605882352941176
E_out médio: 0.2580275229357798

Permutação 6/24
Permutação: (0, 5, 4, 1)
E_in médio: 0.2605882352941176
E_out médio: 0.26146788990825687

Permutação 7/24
Permutação: (1, 0, 4, 5)
E_in médio: 0.26176470588235295
E_out médio: 0.26146788990825687

Permutação 8/24
Permutação: (1, 0, 5, 4)
E_in médio: 0.2605882352941176
E_out médio: 0.2603211009174312

Permutação 9/24
Permutação: (1, 4, 0, 5)
E_in médio: 0.2635294117647059
E_out médio: 0.26146788990825687

Permutação 10/24
Permutação: (1, 4, 5, 0)
E_in 

In [61]:
def classificar_digito(modelos, x):
        for modelo in modelos:
            if modelo['modelo'].predict([x])[0] == 1:
                return modelo['digito']
        return 5

X_train = np.hstack((np.ones((len(train_redu), 1)), train_redu[['i_normalizada', 's_normalizada']].values))
y_train = np.array([1 if y_ == best_permutation[0] else -1 for y_ in train_redu['label']])

X_test = np.hstack((np.ones((len(test_redu), 1)), test_redu[['i_normalizada', 's_normalizada']].values))
y_test = np.array([1 if y_ == best_permutation[0] else -1 for y_ in test_redu['label']])
modelo = LogisticRegression_(eta=0.1, tmax=1000, bs=20)
modelo.fit(X_train, y_train)

# calcular ein e eout
eIn = sum(1 for i in range(len(X_train)) if classificar_digito(best_modelos, X_train[i]) != train_redu['label'].iloc[i])
eOut = sum(1 for i in range(len(X_test)) if classificar_digito(best_modelos, X_test[i]) != test_redu['label'].iloc[i])

print(f"eIn: {eIn / len(X_train)}")
print(f"eOut: {eOut / len(X_test)}")

eIn: 0.2717291857273559
eOut: 0.2878923766816143
